In [1]:
import pinocchio
import numpy as np
from ramp.robot_model import load_robot_model
from ramp.robot_state import RobotState
import pathlib

# Set print options
np.set_printoptions(
    precision=3,        # Number of decimal places
    suppress=True,      # Suppress scientific notation
    linewidth=100,      # Characters per line
    threshold=50        # Show all elements if array size < threshold
)

In [2]:
tcp = "panda_hand_tcp"

In [3]:
rm = load_robot_model(pathlib.Path("robots/panda/configs.toml"))

In [4]:
rs = RobotState.from_random(rm)

In [5]:
J_local = rs.jacobian(tcp, pinocchio.ReferenceFrame.LOCAL)

In [6]:
J_lwa = rs.jacobian(tcp, pinocchio.ReferenceFrame.LOCAL_WORLD_ALIGNED)

In [7]:
J_world = rs.jacobian(tcp, pinocchio.ReferenceFrame.WORLD)

In [8]:
# LOCAL -> WORLD
rs.get_frame_pose(tcp).toActionMatrix() @ J_local

array([[ 0.   , -0.256, -0.195, ...,  0.311,  0.   ,  0.   ],
       [ 0.   , -0.213,  0.235, ...,  0.166,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ..., -0.155,  0.   ,  0.   ],
       [ 0.   , -0.639,  0.706, ..., -0.018,  0.   ,  0.   ],
       [ 0.   ,  0.769,  0.586, ..., -0.664,  0.   ,  0.   ],
       [ 1.   ,  0.   ,  0.397, ..., -0.748,  0.   ,  0.   ]], shape=(6, 9))

In [9]:
J_local

array([[-0.024,  0.048, -0.388, ...,  0.   ,  0.   ,  0.   ],
       [-0.358,  0.21 , -0.169, ...,  0.   ,  0.   ,  0.   ],
       [-0.155,  0.075,  0.102, ...,  0.   ,  0.   ,  0.   ],
       [ 0.601, -0.789,  0.123, ...,  0.   ,  0.   ,  0.   ],
       [ 0.283,  0.358, -0.704, ...,  0.   ,  0.   ,  0.   ],
       [-0.748, -0.499, -0.699, ...,  1.   ,  0.   ,  0.   ]], shape=(6, 9))

In [19]:
J_world

array([[ 0.   , -0.256, -0.195, ...,  0.311,  0.   ,  0.   ],
       [ 0.   , -0.213,  0.235, ...,  0.166,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , ..., -0.155,  0.   ,  0.   ],
       [ 0.   , -0.639,  0.706, ..., -0.018,  0.   ,  0.   ],
       [ 0.   ,  0.769,  0.586, ..., -0.664,  0.   ,  0.   ],
       [ 1.   ,  0.   ,  0.397, ..., -0.748,  0.   ,  0.   ]], shape=(6, 9))

In [11]:
# WORLD -> LOCAL
rs.get_frame_pose(tcp).inverse().toActionMatrix() @ J_world
# W_tcp_B => B_tcp_W => Adj

array([[-0.024,  0.048, -0.388, ...,  0.   ,  0.   ,  0.   ],
       [-0.358,  0.21 , -0.169, ..., -0.   ,  0.   ,  0.   ],
       [-0.155,  0.075,  0.102, ..., -0.   ,  0.   ,  0.   ],
       [ 0.601, -0.789,  0.123, ...,  0.   ,  0.   ,  0.   ],
       [ 0.283,  0.358, -0.704, ...,  0.   ,  0.   ,  0.   ],
       [-0.748, -0.499, -0.699, ...,  1.   ,  0.   ,  0.   ]], shape=(6, 9))

In [12]:
J_lwa

array([[ 0.32 , -0.174, -0.005, ...,  0.   ,  0.   ,  0.   ],
       [ 0.224, -0.144,  0.249, ..., -0.   ,  0.   ,  0.   ],
       [-0.   ,  0.032, -0.358, ...,  0.   ,  0.   ,  0.   ],
       [ 0.   , -0.639,  0.706, ..., -0.018,  0.   ,  0.   ],
       [ 0.   ,  0.769,  0.586, ..., -0.664,  0.   ,  0.   ],
       [ 1.   ,  0.   ,  0.397, ..., -0.748,  0.   ,  0.   ]], shape=(6, 9))

In [16]:
# LOCAL_WORLD_ALIGNED -> LOCAL
b_R_w = rs.get_frame_pose(tcp).rotation.T
np.vstack([b_R_w @ J_lwa[:3, :], b_R_w @ J_lwa[3:, :]]) 
# w_R_b => b_R_w

array([[-0.024,  0.048, -0.388, ...,  0.   ,  0.   ,  0.   ],
       [-0.358,  0.21 , -0.169, ..., -0.   ,  0.   ,  0.   ],
       [-0.155,  0.075,  0.102, ...,  0.   ,  0.   ,  0.   ],
       [ 0.601, -0.789,  0.123, ...,  0.   ,  0.   ,  0.   ],
       [ 0.283,  0.358, -0.704, ...,  0.   ,  0.   ,  0.   ],
       [-0.748, -0.499, -0.699, ...,  1.   ,  0.   ,  0.   ]], shape=(6, 9))

In [68]:
# LOCAL -> LOCAL_WORLD_ALIGNED
pose = rs.get_frame_pose(tcp)
pose.translation = np.zeros(3)
np.vstack([rs.get_frame_pose(tcp).rotation @ J_local[:3, :], rs.get_frame_pose(tcp).rotation @ J_local[3:, :]]) # Or
pose.toActionMatrix() @ J_local

array([[ 0.32 , -0.174, -0.005, ...,  0.   ,  0.   ,  0.   ],
       [ 0.224, -0.144,  0.249, ...,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.032, -0.358, ...,  0.   ,  0.   ,  0.   ],
       [ 0.   , -0.639,  0.706, ..., -0.018,  0.   ,  0.   ],
       [ 0.   ,  0.769,  0.586, ..., -0.664,  0.   ,  0.   ],
       [ 1.   ,  0.   ,  0.397, ..., -0.748,  0.   ,  0.   ]], shape=(6, 9))

In [69]:
# WORLD -> LOCAL_WORLD_ALIGNED
pose = rs.get_frame_pose(tcp)
pose.rotation = np.eye(3)
np.block([[      np.eye(3), -pinocchio.skew(rs.get_frame_pose(tcp).translation)],
          [np.zeros((3, 3)),                                          np.eye(3)]]) @ J_world # Or
pose.inverse().toActionMatrix() @ J_world

array([[ 0.32 , -0.174, -0.005, ...,  0.   ,  0.   ,  0.   ],
       [ 0.224, -0.144,  0.249, ..., -0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.032, -0.358, ..., -0.   ,  0.   ,  0.   ],
       [ 0.   , -0.639,  0.706, ..., -0.018,  0.   ,  0.   ],
       [ 0.   ,  0.769,  0.586, ..., -0.664,  0.   ,  0.   ],
       [ 1.   ,  0.   ,  0.397, ..., -0.748,  0.   ,  0.   ]], shape=(6, 9))

In [30]:
# LOCAL_WORLD_ALIGNED -> WORLD
tcp_pose = rs.get_frame_pose(tcp)
# First convert to LOCAL, then to WORLD
# R = I
J_local_from_lwa = np.vstack([tcp_pose.rotation.T @ J_lwa[:3, :], 
                              tcp_pose.rotation.T @ J_lwa[3:, :]])
tcp_pose.toActionMatrix() @ J_local_from_lwa # Or
np.block([[      np.eye(3), pinocchio.skew(tcp_pose.translation)],
          [np.zeros((3, 3)),                           np.eye(3)]]) @ J_lwa

array([[ 0.   , -0.256, -0.195, ...,  0.311,  0.   ,  0.   ],
       [ 0.   , -0.213,  0.235, ...,  0.166,  0.   ,  0.   ],
       [ 0.   , -0.   ,  0.   , ..., -0.155,  0.   ,  0.   ],
       [ 0.   , -0.639,  0.706, ..., -0.018,  0.   ,  0.   ],
       [ 0.   ,  0.769,  0.586, ..., -0.664,  0.   ,  0.   ],
       [ 1.   ,  0.   ,  0.397, ..., -0.748,  0.   ,  0.   ]], shape=(6, 9))

In [2]:
# Twists are elements of the Lie algebra se(3) which is the tangent space of the special Euclidean group SE(3) of rigid body transformations.
# Angular velocities are themselves elements of the Lie algebra so(3), which is the tangent space of the special orthogonal group SO(3) of orientations.
# We can integrate twists into transformations via the exponential map, which generalizes both the integration of linear velocities into positions and of angular velocities into orientations. 
X = pinocchio.SE3.Random()
twist = pinocchio.Motion.Random()

In [17]:
twist.homogeneous

array([[ 0.   ,  0.514, -0.967,  0.271],
       [-0.514,  0.   , -0.214,  0.435],
       [ 0.967,  0.214,  0.   , -0.717],
       [ 0.   ,  0.   ,  0.   ,  0.   ]])

In [13]:
pinocchio.log6(X).action

array([[ 0.   , -1.092, -1.242,  0.   ,  0.053,  1.069],
       [ 1.092,  0.   , -1.194, -0.053,  0.   , -1.008],
       [ 1.242,  1.194,  0.   , -1.069,  1.008,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   , -1.092, -1.242],
       [ 0.   ,  0.   ,  0.   ,  1.092,  0.   , -1.194],
       [ 0.   ,  0.   ,  0.   ,  1.242,  1.194,  0.   ]])

In [14]:
X.action

array([[ 0.045, -0.995, -0.088,  0.936, -0.028,  0.796],
       [-0.04 ,  0.087, -0.995,  0.011, -0.83 , -0.073],
       [ 0.998,  0.049, -0.036, -0.042,  0.902,  0.053],
       [ 0.   ,  0.   ,  0.   ,  0.045, -0.995, -0.088],
       [ 0.   ,  0.   ,  0.   , -0.04 ,  0.087, -0.995],
       [ 0.   ,  0.   ,  0.   ,  0.998,  0.049, -0.036]])

In [35]:
# From manifold (SE3) to tangent space (Motion)
pinocchio.log6(X).vector # Log(X) = log(X)v (vee)

array([ 1.008,  1.069, -0.053,  1.194, -1.242,  1.092])

In [36]:
# From tangent space (Motion) to manifold (SE3)
# integrate twists into transformations
pinocchio.exp6(twist) # Exp(theta) = exp(theta^)

SE3(array([[ 0.46 ,  0.321, -0.828,  0.631],[-0.507,  0.86 ,  0.052,  0.355],[ 0.729,  0.396,  0.558, -0.417],[ 0.   ,  0.   ,  0.   ,  1.   ]]))

In [37]:
thetahat = twist.homogeneous # theta^
thetahat

array([[ 0.   ,  0.514, -0.967,  0.271],
       [-0.514,  0.   , -0.214,  0.435],
       [ 0.967,  0.214,  0.   , -0.717],
       [ 0.   ,  0.   ,  0.   ,  0.   ]])

In [21]:
pinocchio.skew(twist.angular)

array([[ 0.   ,  0.514, -0.967],
       [-0.514,  0.   , -0.214],
       [ 0.967,  0.214,  0.   ]])

In [18]:
pinocchio.Jexp6(twist)

array([[ 0.812, -0.264,  0.418,  0.016, -0.349, -0.242],
       [ 0.199,  0.951,  0.174,  0.296, -0.134,  0.196],
       [-0.453, -0.018,  0.846,  0.15 , -0.049,  0.114],
       [ 0.   ,  0.   ,  0.   ,  0.812, -0.264,  0.418],
       [ 0.   ,  0.   ,  0.   ,  0.199,  0.951,  0.174],
       [ 0.   ,  0.   ,  0.   , -0.453, -0.018,  0.846]])

In [28]:
X.action # Adjoint matrix of SE3. See example 6

array([[ 0.045, -0.995, -0.088,  0.936, -0.028,  0.796],
       [-0.04 ,  0.087, -0.995,  0.011, -0.83 , -0.073],
       [ 0.998,  0.049, -0.036, -0.042,  0.902,  0.053],
       [ 0.   ,  0.   ,  0.   ,  0.045, -0.995, -0.088],
       [ 0.   ,  0.   ,  0.   , -0.04 ,  0.087, -0.995],
       [ 0.   ,  0.   ,  0.   ,  0.998,  0.049, -0.036]])

In [32]:
np.block([[      X.rotation, pinocchio.skew(X.translation) @ X.rotation],
          [np.zeros((3, 3)),                                 X.rotation]])

array([[ 0.045, -0.995, -0.088,  0.936, -0.028,  0.796],
       [-0.04 ,  0.087, -0.995,  0.011, -0.83 , -0.073],
       [ 0.998,  0.049, -0.036, -0.042,  0.902,  0.053],
       [ 0.   ,  0.   ,  0.   ,  0.045, -0.995, -0.088],
       [ 0.   ,  0.   ,  0.   , -0.04 ,  0.087, -0.995],
       [ 0.   ,  0.   ,  0.   ,  0.998,  0.049, -0.036]])

In [30]:
pinocchio.skew(X.translation) @ X.rotation

array([[ 0.936, -0.028,  0.796],
       [ 0.011, -0.83 , -0.073],
       [-0.042,  0.902,  0.053]])

In [33]:
X.toActionMatrix()

array([[ 0.045, -0.995, -0.088,  0.936, -0.028,  0.796],
       [-0.04 ,  0.087, -0.995,  0.011, -0.83 , -0.073],
       [ 0.998,  0.049, -0.036, -0.042,  0.902,  0.053],
       [ 0.   ,  0.   ,  0.   ,  0.045, -0.995, -0.088],
       [ 0.   ,  0.   ,  0.   , -0.04 ,  0.087, -0.995],
       [ 0.   ,  0.   ,  0.   ,  0.998,  0.049, -0.036]])

In [29]:
X.rotation

array([[ 0.045, -0.995, -0.088],
       [-0.04 ,  0.087, -0.995],
       [ 0.998,  0.049, -0.036]])

In [70]:
pinocchio.Jlog6(rs.get_frame_pose(tcp))

array([[ 0.805, -0.511, -0.582, -0.082, -0.216, -0.029],
       [-0.017,  0.574, -1.1  ,  0.051, -0.146, -0.18 ],
       [ 0.774,  0.974,  0.424,  0.165,  0.089, -0.177],
       [ 0.   ,  0.   ,  0.   ,  0.805, -0.511, -0.582],
       [ 0.   ,  0.   ,  0.   , -0.017,  0.574, -1.1  ],
       [ 0.   ,  0.   ,  0.   ,  0.774,  0.974,  0.424]])

In [19]:
pinocchio.Jlog6(X)

array([[ 0.754, -0.679, -0.504,  0.25 ,  0.029,  0.627],
       [ 0.413,  0.765, -0.719, -0.023, -0.204, -0.393],
       [ 0.738,  0.475,  0.734, -0.442,  0.616,  0.024],
       [ 0.   ,  0.   ,  0.   ,  0.754, -0.679, -0.504],
       [ 0.   ,  0.   ,  0.   ,  0.413,  0.765, -0.719],
       [ 0.   ,  0.   ,  0.   ,  0.738,  0.475,  0.734]])

In [77]:
J_local

array([[-0.024,  0.048, -0.388, ...,  0.   ,  0.   ,  0.   ],
       [-0.358,  0.21 , -0.169, ...,  0.   ,  0.   ,  0.   ],
       [-0.155,  0.075,  0.102, ...,  0.   ,  0.   ,  0.   ],
       [ 0.601, -0.789,  0.123, ...,  0.   ,  0.   ,  0.   ],
       [ 0.283,  0.358, -0.704, ...,  0.   ,  0.   ,  0.   ],
       [-0.748, -0.499, -0.699, ...,  1.   ,  0.   ,  0.   ]], shape=(6, 9))

In [8]:
pinocchio.Jlog6(rs.get_frame_pose(tcp)) # jacs_left_right: Log(X) & Right Jacobians 

array([[ 0.819,  0.713, -0.184,  0.022,  0.006,  0.219],
       [-0.698,  0.576, -0.826,  0.087,  0.125,  0.118],
       [-0.235,  0.813,  0.756, -0.104, -0.2  ,  0.097],
       [ 0.   ,  0.   ,  0.   ,  0.819,  0.713, -0.184],
       [ 0.   ,  0.   ,  0.   , -0.698,  0.576, -0.826],
       [ 0.   ,  0.   ,  0.   , -0.235,  0.813,  0.756]])

In [25]:
np.linalg.inv(pinocchio.Jexp6(pinocchio.log6(rs.get_frame_pose(tcp))))

array([[ 0.819,  0.713, -0.184,  0.022,  0.006,  0.219],
       [-0.698,  0.576, -0.826,  0.087,  0.125,  0.118],
       [-0.235,  0.813,  0.756, -0.104, -0.2  ,  0.097],
       [ 0.   ,  0.   ,  0.   ,  0.819,  0.713, -0.184],
       [-0.   , -0.   , -0.   , -0.698,  0.576, -0.826],
       [ 0.   ,  0.   ,  0.   , -0.235,  0.813,  0.756]])

In [11]:
motion = pinocchio.Motion.Random()

In [19]:
motion.homogeneous

array([[ 0.   ,  0.037,  0.829,  0.471],
       [-0.037,  0.   , -0.098,  0.456],
       [-0.829,  0.098,  0.   , -0.081],
       [ 0.   ,  0.   ,  0.   ,  0.   ]])